## **Construyendo aplicaciones potenciadas por LLMs con LangChain**

In [1]:
import dotenv
import sys
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

**Introducción**

LangChain está diseñado para ayudar a los desarrolladores a crear aplicaciones end-to-end utilizando modelos de lenguaje. Ofrece una serie de herramientas, componentes e interfaces que simplifican el proceso de creación de aplicaciones basadas en LLMs y modelos especializados en chat. LangChain agiliza la gestión de las interacciones con los LLM, el encadenamiento (o las chains) de múltiples componentes y la integración de recursos adicionales, como API, bases de datos y el acceso a internet. 

**Usos de caso de Prompts**

Una de las características clave de LangChain es su soporte para los prompts, lo cual permite la gestión de prompts, la optimización de prompt y una interfaz generica para cualquier LLM. El framework también proporciona utilidades comunes para trabajar con modelos de lenguaje.

ChatPromptTemplate se utiliza para crear una conversación estructurada con el modelo de IA, facilitando la gestión del flujo y el contenido de la conversación. En LangChain, las plantillas de prompts de mensajes se utilizan para construir y trabajar con prompts, lo que nos permite explotar al máximo el potencial del modelo de chat específico.

Los mensajes System y Human difieren en sus roles y propósitos cuando se interactúa con modelos de chat. SystemMessagePromptTemplate proporciona instrucciones iniciales, contexto o datos para el modelo de IA, mientras que HumanMessagePromptTemplate son mensajes del usuario a los que responde el modelo de IA.

Veamos un ejemplo para ilustrar lo anterior:

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Instanciamos el LLM a utilizar
chat = ChatOpenAI(
    engine="gpt-35-turbo",
    temperature=0
)
template = "Eres un asistente que ayuda a los usuarios encontrar información sobre IA"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Encuentra información acerca de {topic}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Pasamos los mensajes de Sistema y mensaje de Usuario
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

response = chat(chat_prompt.format_prompt(topic="Computer vision").to_messages())
print(response.content)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


La visión por computadora (Computer Vision en inglés) es una rama de la inteligencia artificial que se enfoca en permitir a las computadoras interpretar y comprender el mundo visual. Esta tecnología se utiliza en una amplia variedad de aplicaciones, desde la detección de objetos en imágenes hasta la conducción autónoma de vehículos.

Algunas de las técnicas utilizadas en la visión por computadora incluyen el procesamiento de imágenes, el aprendizaje profundo y la detección de características. Estas técnicas permiten a las computadoras analizar y comprender imágenes y videos, identificar objetos y patrones, y tomar decisiones basadas en esa información.

La visión por computadora se utiliza en una amplia variedad de aplicaciones, incluyendo la seguridad y vigilancia, la medicina, la robótica, la automatización industrial, la realidad aumentada y la realidad virtual, entre otras.


El uso del método to_messages en LangChain permite convertir el valor formateado de un modelo de mensaje de chat en una lista de objetos de mensaje. Esto resulta útil cuando se trabaja con modelos de chat, ya que proporciona una forma estructurada de gestionar la conversación y garantiza que el modelo de chat pueda comprender el contexto y las funciones de los mensajes,

#### **Ejemplo de Chain de Resumen**

Los prompts de LangChain pueden encontrarse en varios casos de uso, como cadenas de resumen o de pregunta-respuesta. Por ejemplo, al crear una cadena de resumen, LangChain permite interactuar con una fuente de datos externa para obtener datos que se utilizarán en el paso de generación. Esto podría implicar resumir un texto extenso o responder preguntas utilizando fuentes de datos específicas.

El siguiente código inicializará el modelo de lenguaje utilizando la clase OpenAI con una temperatura de 0 - porque queremos una salida determinista.  La función load_summarize_chain acepta una instancia del modelo de lenguaje y devuelve una cadena de resumen pre-construida. Por último, la clase PyPDFLoader se encarga de cargar archivos PDF y convertirlos a un formato adecuado para ser procesados por LangChain. 

Es importante tener en cuenta que es necesario instalar el paquete pypdf para ejecutar el siguiente código. Aunque es muy recomendable instalar las últimas versiones de este paquete, los códigos han sido probados en la versión 3.10.0. 

In [4]:
#!pip install pypdf

In [6]:
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain  
from langchain.document_loaders import PyPDFLoader

# Instanciamos el LLM
llm = OpenAI(
    engine="text-davinci-003",
    temperature=0
)

# Summarization chain
summarize_chain = load_summarize_chain(llm)

# Cargamos el documento PDF a resumir
document_loader = PyPDFLoader(file_path="./docs/LLMs.pdf")
document = document_loader.load()

# Resumimos el documento
summary = summarize_chain(document)
print(summary["output_text"])

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


 Los modelos de lenguaje generativos son una forma de inteligencia artificial que ha revolucionado la forma en que interactuamos con la tecnología y cómo se produce el contenido escrito. Estos modelos se basan en algoritmos de aprendizaje automático y redes neuronales, y han sido ampliamente utilizados en diversas aplicaciones. Estos modelos tienen la capacidad de generar texto desde cero, responder preguntas, traducir idiomas y simular conversaciones humanas. Sin embargo, también hay desafíos éticos y técnicos que deben abordarse para asegurar que estos modelos sean seguros, equitativos y beneficiosos para la sociedad.


En este ejemplo, el código utiliza la cadena de integración por defecto proporcionada por la función load_summarize_chain. Sin embargo, puede personalizar el proceso de integración proporcionando Prompt Templates.

#### **Ejemplo de pregunta-respuesta**

También podemos utilizar LangChain para gestionar preguntas generales de los LLM. Estos modelos son competentes a la hora de abordar cuestiones fundamentales. Sin embargo, es crucial tener en cuenta el posible problema de las alucinaciones, ya que los modelos pueden generar información no coherente. 

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

prompt = PromptTemplate(
    template="Question: {question}\Answer:",
    input_variables=["question"]
)
llm = OpenAI(engine="text-davinci-003", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


Definimos un template de pregunta personalizada creando una instancia de la clase PromptTemplate. La cadena del template contiene un marcador de posición {question} para la pregunta de entrada, seguido de un carácter de nueva línea y la etiqueta "Answer:". El argumento input_variables se establece en la lista de argumentos de posición disponibles en el prompt (como una pregunta en este caso) para indicar el nombre de la variable que la cadena reemplazará en el método template.run().

Luego, instanciamos un modelo OpenAI llamado text-davinci-003 con una temperatura de 0. Se utiliza la clase OpenAI para crear la instancia, y se proporcionan los argumentos model_name y temperature. Por último, creamos una cadena de pregunta-respuesta utilizando la clase LLMChain. 

El constructor de la clase toma dos argumentos: llm, que es el modelo OpenAI instanciado, y prompt, que es la plantilla personalizada de prompt que definimos anteriormente.

In [9]:
print(chain.run("Las inteligencias artificiales nos dominarán?"))



No hay forma de saber con certeza si las inteligencias artificiales nos dominarán en el futuro. Sin embargo, hay muchos expertos que creen que la inteligencia artificial no será una amenaza para la humanidad, sino más bien una herramienta para ayudarnos a mejorar nuestras vidas.


Este ejemplo muestra cómo LangChain simplifica la integración de LLM con fuentes de datos personalizadas y plantillas de instrucciones para aplicaciones de respuesta a preguntas. Para crear aplicaciones NLP más avanzadas, puede ampliar este ejemplo para incluir otros componentes, como la generación aumentada de datos, agentes o funciones de memoria.

El soporte de LangChain para secuencias de cadenas también permite a los desarrolladores crear aplicaciones más complejas con múltiples llamadas a LLMs u otras utilidades. Estas cadenas pueden servir para varios propósitos: asistentes personales, chatbots, consulta de datos tabulares, interacción con APIs, extracción, evaluación y resumen.